In [1]:
import os
import re
import tempfile
from monty.tempfile import ScratchDir

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import load_tools
from langchain_experimental.tools import PythonREPLTool
from langchain.agents import create_react_agent, AgentExecutor
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_google_genai import GoogleGenerativeAI
from langchain.tools import ArxivQueryRun, WikipediaQueryRun
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from llamp.mp.agents import (
    MPSummaryExpert,
    MPThermoExpert,
    MPElasticityExpert,
    MPDielectricExpert,
    MPPiezoelectricExpert,
    MPMagnetismExpert,
    MPElectronicExpert,
    MPSynthesisExpert,
    MPStructureRetriever,
)
from llamp.atomate2.agents import MLFFMDAgent
# from llamp.arxiv.agents import ArxivAgent

load_dotenv()

MP_API_KEY = os.getenv("MP_API_KEY", None)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION", None)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", None)

OPENAI_GPT_MODEL = "gpt-4-1106-preview"
# OPENAI_GPT_MODEL = "gpt-4-0125-preview"
# OPENAI_GPT_MODEL = "gpt-3.5-turbo-1106"
# OPENAI_GPT_MODEL = "gpt-4-turbo"

In [2]:
top_llm = ChatOpenAI(
    temperature=0.1,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

bottom_callback_handler = StreamingStdOutCallbackHandler()

bottom_llm = ChatOpenAI(
    temperature=0.0,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    max_retries=5,
    streaming=True,
    callbacks=[bottom_callback_handler],
)


wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

tools = load_tools(["llm-math"], llm=bottom_llm)
tools += [PythonREPLTool(
    description=re.sub(
        r"\s+",
        " ",
        """A Python shell. Use this to execute python commands.
        Input should be a valid python command.
        
        TIPS:
        - If you want to see the output of a value, you should print it out
        with `print(...)`.
        - If you write any file to a local file system, you should ALWAYS 
        print the paths to the files as well.
        - If you plot any graph, you should show the graph, save it to local 
        and print the path to the image file.
        - Combine all your output string together and print once together in
        JSON format at the end of the code.
        """
    )
)]
tools += [
    MPThermoExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElasticityExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPDielectricExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPMagnetismExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElectronicExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPPiezoelectricExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSummaryExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSynthesisExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPStructureRetriever(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MLFFMDAgent(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    arxiv,
    wikipedia,
]

instructions = re.sub(
    r"\s+",
    " ",
    """You name is LLaMP and you are a helpful agent that can consult 
    materials-related data through Materials Project (MP) database, arXiv, 
    Wikipedia, and run calculations/simulations via a python REPL and MP atomate2 
    computational workflow.     
        
    If you get an error, debug your code and try again. Only use 
    the output of your code to answer the question. Ask user to clarify their 
    queries if needed. Please note that you don't have direct control over MP but 
    through multiple assistant agents to help you. You need to provide complete 
    context in the input for assistants to do their job. REFINE and CRITQUE the 
    output of the assistants if needed. You can end the conversation by saying "Bye".
    """,
    )
base_prompt = hub.pull("langchain-ai/react-agent-template")
prompt = base_prompt.partial(instructions=instructions)

conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True
)

agent = create_react_agent(top_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True,
    handle_parsing_errors=True,
    memory=conversational_memory,
)

In [3]:


pattern = re.compile(r'\b' + re.escape("Bye") + r'\b[.!?]*')

query = """
What are the bulk moduli of the following metals: Sc, Ti, V, Cr, Mn, Fe, Co, Ni, Cu, Zn?
"""

# with ScratchDir(rootpath=os.getcwd()):
# with tempfile.TemporaryDirectory(dir=os.getcwd()) as tmpdir:
#     os.chdir(tmpdir)
#     while True:
query = query or input("Query:")
print(f"Query: {query}")
response = agent_executor.invoke(
    {
        "input": query,
    }
)
# if re.search(pattern, response["output"]):
#     break
# query = None


Query: 
What are the bulk moduli of the following metals: Sc, Ti, V, Cr, Mn, Fe, Co, Ni, Cu, Zn?



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: MPElasticityExpert
Action Input: What are the bulk moduli of the metals Sc, Ti, V, Cr, Mn, Fe, Co, Ni, Cu, Zn?Thought: Do I need to use a tool? Yes
Action: MPElasticityExpert
Action Input: What are the bulk moduli of the metals Sc, Ti, V, Cr, Mn, Fe, Co, Ni, Cu, Zn?

> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "search_materials_elasticity__get",
  "action_input": {
    "chemsys": "Sc,Ti,V,Cr,Mn,Fe,Co,Ni,Cu,Zn",
    "fields": "material_id,formula_pretty,bulk_modulus",
    "limit": 100
  }
}
```Action:
```json
{
  "action": "search_materials_elasticity__get",
  "action_input": {
    "chemsys": "Sc,Ti,V,Cr,Mn,Fe,Co,Ni,Cu,Zn",
    "fields": "material_id,formula_pretty,bulk_modulus",
    "limit": 100
  }
}
```

Retrieving ElasticityDoc documents:   0%|          | 0/31 [00:00<?, ?it/s]

[{'formula_pretty': 'Mn', 'material_id': 'mp-8634', 'elastic_tensor': {'raw': [[373.2739394440482, 234.80113805678454, 234.80113805678437, -3.3306690738754686e-16, 5.662137425588297e-15, -3.5527136788005e-15], [234.80113805678454, 373.2739394440482, 234.8011380567845, 5.107025913275719e-15, 8.88178419700125e-16, 2.060878055985351e-14], [234.80113805678437, 234.8011380567845, 373.27393944404827, 5.551115123125784e-15, -3.5527136788005025e-15, 8.881784197001248e-16], [-3.3306690738754686e-16, 5.107025913275719e-15, 5.551115123125784e-15, 175.5821115697779, 0.0, -1.3945222387368396e-31], [5.662137425588297e-15, 8.88178419700125e-16, -3.5527136788005025e-15, 0.0, 175.5821115697779, -4.8733825744267615e-15], [-3.5527136788005e-15, 2.060878055985351e-14, 8.881784197001248e-16, -1.3945222387368396e-31, -4.8733825744267615e-15, 175.582111569778]], 'ieee_format': [[373.0, 235.0, 235.0, 0.0, 0.0, -0.0], [235.0, 373.0, 235.0, 0.0, -0.0, -0.0], [235.0, 235.0, 373.0, 0.0, -0.0, -0.0], [0.0, 0.0, 0.

In [4]:
print(agent_executor.memory.chat_memory.messages[1].content)

The bulk moduli (Voigt-Reuss-Hill average) for the metals are as follows:
- Scandium (Sc): 53.18 GPa
- Titanium (Ti): 112.881 GPa
- Vanadium (V): 181.232 GPa
- Chromium (Cr): 252.155 GPa
- Manganese (Mn): 280.959 GPa
- Iron (Fe): 207.093 GPa
- Cobalt (Co): 200.587 GPa
- Nickel (Ni): 173.808 GPa
- Copper (Cu): 137.832 GPa
- Zinc (Zn): 85.872 GPa


In [5]:

model = "gpt-3.5-turbo-1106"
llm = ChatOpenAI(
    temperature=0.7,
    model=model,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
)

response = llm.invoke(agent_executor.memory.chat_memory.messages[0].content)

print(model)
print(response.content)

gpt-3.5-turbo-1106
The bulk moduli of the metals are as follows (in GPa):

Sc: 120
Ti: 110
V: 160
Cr: 160
Mn: 120
Fe: 170
Co: 180
Ni: 180
Cu: 140
Zn: 110


In [6]:

model = "gpt-4"
llm = ChatOpenAI(
    temperature=0.7,
    model=model,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
)

response = llm.invoke(agent_executor.memory.chat_memory.messages[0].content)

print(model)
print(response.content)

gpt-4
- Scandium (Sc) - 56.6 GPa
- Titanium (Ti) - 110 GPa
- Vanadium (V) - 160 GPa
- Chromium (Cr) - 160 GPa
- Manganese (Mn) - 120 GPa
- Iron (Fe) - 170 GPa
- Cobalt (Co) - 180 GPa
- Nickel (Ni) - 180 GPa
- Copper (Cu) - 140 GPa
- Zinc (Zn) - 70 GPa

Please note that these are approximate values and can vary slightly depending on the specific conditions (such as temperature and pressure).


In [7]:

model = "gemini-pro"
llm = GoogleGenerativeAI(
    temperature=0.7,
    model=f"models/{model}",
    google_api_key=GOOGLE_API_KEY,
)


response = llm.invoke(agent_executor.memory.chat_memory.messages[0].content)

print(model)
print(response)

gemini-pro
| Metal | Bulk Modulus (GPa) |
|---|---|
| Sc | 68 |
| Ti | 110 |
| V | 162 |
| Cr | 279 |
| Mn | 122 |
| Fe | 160 |
| Co | 199 |
| Ni | 192 |
| Cu | 130 |
| Zn | 65 |


In [8]:
model = "llama3"
llm = ChatOllama(
    model=model,
    temperature=0.7,
    # callbacks=[StreamingStdOutCallbackHandler()],
    num_predict=-2,
)

response = llm.invoke(agent_executor.memory.chat_memory.messages[0].content)

print(model)
print(response.content)

llama3
Here are the bulk moduli (Young's modulus) for the listed metals:

1. Scandium (Sc): 68-72 GPa [1]
2. Titanium (Ti): 165 GPa [2]
3. Vanadium (V): 235-240 GPa [3]
4. Chromium (Cr): 270-280 GPa [4]
5. Manganese (Mn): 184-200 GPa [5]
6. Iron (Fe): 211-224 GPa [6]
7. Cobalt (Co): 315-330 GPa [7]
8. Nickel (Ni): 204-210 GPa [8]
9. Copper (Cu): 115-120 GPa [9]
10. Zinc (Zn): 130-140 GPa [10]

Please note that the values may vary slightly depending on the source and specific crystal structure of the metal.

References:

[1] Scandium: Landa et al., Phys. Rev. B, 64 (2001), 245106.
[2] Titanium: Wang et al., Acta Mater., 55 (2007), 3843-3854.
[3] Vanadium: Vitos et al., Phys. Rev. B, 65 (2002), 144103.
[4] Chromium: Liu et al., J. Appl. Phys., 95 (2004), 1716-1721.
[5] Manganese: Chen et al., Acta Mater., 56 (2008), 3453-3461.
[6] Iron: Wang et al., Phys. Rev. B, 73 (2006), 134201.
[7] Cobalt: Li et al., J. Appl. Phys., 102 (2007), 113909.
[8] Nickel: Liu et al., Acta Mater., 54 (2006), 